<a href="https://colab.research.google.com/github/itissandeep98/StackOverFlow-Tag-Predictor/blob/master/datasetGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
import numpy as np
import pandas as pd 
import csv,nltk,random,pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer,WordNetLemmatizer,SnowballStemmer
import re
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split,ShuffleSplit,GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
import pickle
import joblib
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
af=pd.read_csv('/content/drive/MyDrive/Ml Project/Train.zip')

# Pre-Process Data

In [ ]:
def generate(n):
  print(n)
  
  df=af[:n]

  duplicate_pairs = df.sort_values('Title', ascending=False).duplicated('Title')
  df = df[~duplicate_pairs]

  df["Tags"]=df["Tags"].astype(str)
  df["tag_count"] = df["Tags"].apply(lambda x : len(x.split()))
  qus_list=[]

  for index,row in df.iterrows():
      title, body, tags = row["Title"], row["Body"], row["Tags"]   
      body=re.sub('<code>(.*?)</code>', '', body, flags=re.MULTILINE|re.DOTALL)
      body = re.sub('<.*?>', ' ', str(body.encode('utf-8')))
      title=title.encode('utf-8')
      question=str(title)+" "+str(body)
      question=re.sub(r'[^A-Za-z]+',' ',question)
      words=word_tokenize(str(question.lower()))
      stop_words = set(stopwords.words('english'))
      stemmer = SnowballStemmer("english")
      question=' '.join(str(stemmer.stem(j)) for j in words if j not in stop_words and (len(j)!=1 or j=='c'))
      qus_list.append(question)
    
  df["question"] = qus_list
  preprocessed_df = df[["question","Tags"]]
  preprocessed_df.to_csv("preprocessed_"+str(n)+"k.csv")

In [ ]:
for i in range(10000,100001,10000):
  generate(i)
  !cp /content/preprocessed_"$i"k.csv /content/drive/MyDrive/MlProject/preprocessed_data/

# Vectorised Data

In [ ]:
def tags_to_consider(n,multilabel_y):
    tag_i_sum = multilabel_y.sum(axis=0).tolist()[0]
    sorted_tags_i = sorted(range(len(tag_i_sum)), key=lambda i: tag_i_sum[i], reverse=True)
    yn_multilabel=multilabel_y[:,sorted_tags_i[:n]]
    return yn_multilabel

In [ ]:
def vectorize(n):
  print(n)
  preprocessed_df=pd.read_csv("/content/preprocessed_"+str(n)+"k.csv")
  preprocessed_df["tag_count"] = preprocessed_df["Tags"].apply(lambda text: len(text.split(" ")))
  vectorizer = CountVectorizer(tokenizer = lambda x: x.split())
  multilabel_y = vectorizer.fit_transform(preprocessed_df['Tags'])
  yx_multilabel = tags_to_consider(50,multilabel_y)
  preprocessed_df.to_csv("vectorised_X_"+str(n)+"k.csv")
  joblib.dump(yx_multilabel, 'vectorised_y_'+str(n)+'.pkl') 


In [ ]:
for i in range(10000,100001,10000):
  vectorize(i)
  !cp /content/vectorised_X_"$i"k.csv /content/drive/MyDrive/MlProject/data/
  !cp /content/vectorised_y_"$i".pkl /content/drive/MyDrive/MlProject/data/